In [1]:
import requests  # Downloading the file
import psycopg2  # Connect to PostgreSQL
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv  # load .env file with DB Credentials
from tqdm import tqdm  # Progress Bar while downloading
from io import StringIO


def make_connection():
    """ 
    Connect to the PostgreSQL database server 
    """
    
    load_dotenv()

    DB_NAME = os.getenv('DB_NAME')
    DB_USER = os.getenv('DB_USER')
    DB_PASSWORD = os.getenv('DB_PASSWORD')
    HOST = os.getenv('HOST')
    PORT = os.getenv('PORT')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(host = HOST, 
                                      database = DB_NAME, 
                                      user = DB_USER, 
                                      password = DB_PASSWORD, 
                                      port = PORT)
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error while connecting to PostgreSQL:\n{error}')
        return -1
    
    print("Connection successful")
        
    return connection


def create_tables(conn):
    
    queries = (
         """ 
        CREATE TABLE IF NOT EXISTS countries_apple (
            region varchar(48),
            geo_type varchar(14),
            alternative_name varchar(85),
            sub_region varchar(33),
            country varchar(20)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS mobility_stats_apple (
            region varchar(48),
            date date,
            driving numeric(6,2),
            transit numeric(6,2),
            walking numeric(6,2),
            
            PRIMARY KEY (region, date)
        )
        """
    )
    try: 
        with conn.cursor() as cursor:
            for query in queries:
                cursor.execute(query)
            conn.commit()
            print("Tables created successfully!\n----------------------------")
            
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error while creating tables!\nRolling back changes...\n', error)
        conn.rollback()
        return -1

    
def rearrange_df(df):
    
    """ 
    1. Convert columns to rows
    2. Convert rows to columns
    3. Drop level to get one level of columns
    4. Rename axis to remove transportation_type which is our previous column name
    5. Reset index to convert our indices to columns
    """
    
    inter_df = pd.melt(frame=df, 
                       id_vars=['region', 'transportation_type'], 
                       value_vars=df.columns[2:], 
                       var_name='date', 
                       value_name='percent_change_from_baseline'
                      )
    inter_df = pd.pivot_table(inter_df, columns='transportation_type', index=['region', 'date'])
    
    inter_df.columns = inter_df.columns.droplevel()
    inter_df = inter_df.rename_axis(None, axis=1)
    inter_df = inter_df.reset_index()

    return inter_df


def import_data(conn, table_name, df):
    
    buffer = StringIO()
    df.to_csv(buffer, header=False, index=False)
    buffer.seek(0)


    with conn.cursor() as cursor:        
        try:
            cursor.execute(f'TRUNCATE {table_name} CASCADE;')
            print(f'Truncated {table_name}')

            df.where(pd.notnull(df), None)
            
            cursor.copy_expert(f"COPY {table_name} from STDIN CSV QUOTE '\"'", buffer)
            conn.commit()
            print("Done!\n-------------------------------")
            return 1

        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            return -1

        
def driver_code():
    
    count = 0
    
    file_path = '../Data'
    
    for file in os.listdir('../Data'):
        if file.startswith('applemobilitytrends'):
            file_path = os.path.join(file_path ,file)

    conn = make_connection()
    create_tables(conn)

    df = pd.read_csv(filepath_or_buffer=file_path, low_memory=False)

    dates = df[df.columns[6:]]
    nulls = dates[dates.isna().all(axis=1)].index
    df.drop(index=nulls, inplace=True)

    countries_util = df[['geo_type', 'region', 'alternative_name', 'sub-region', 'country']].drop_duplicates()

    countries_util = countries_util[['region', 'geo_type', 'alternative_name', 'sub-region', 'country']]

    df = df.drop(labels=['geo_type', 'alternative_name', 'country', 'sub-region'], axis=1)
    df.head()

    inter_df = rearrange_df(df)

    count += import_data(conn, 'countries_apple', countries_util)
    count += import_data(conn, 'mobility_stats_apple', inter_df)
    
    return count, file_path

In [4]:
count, file_path = driver_code()
print(count)

Connecting to the PostgreSQL database...
Connection successful
Tables created successfully!
----------------------------
Truncated countries_apple
Done!
-------------------------------
Truncated mobility_stats_apple
Done!
-------------------------------
2


In [5]:
os.remove(file_path)